In [1]:
import time
import urllib
import requests
import json
from splinter import Browser
from selenium import webdriver
from config import client_id, username, password

In [2]:
# path to chrome driver
executable_path = {'executable_path': "/usr/local/bin/chromedriver"}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# url components
method = 'GET'
url = 'https://auth.tdameritrade.com/auth?'
client_code = client_id + '@AMER.OAUTHAP'
payload = {'response_type': 'code', 'redirect_uri': 'https://127.0.0.1', 'client_id': client_code}

# build url
built_url = requests.Request(method, url, params=payload).prepare()
built_url = built_url.url

# go to url
browser.visit(built_url)

# define login credentials
payload = {'username': username, 'password': password}

# fill out login
browser.find_by_id('username').first.fill(payload['username'])
browser.find_by_id('password').first.fill(payload['password'])
browser.find_by_id('accept').first.click()

# accept terms and conditions
browser.find_by_id('accept').first.click()

# allow page to load
time.sleep(1)
new_url = browser.url

# grab url and parse
parse_url = urllib.parse.unquote(new_url.split('code=')[1])

# close browser
browser.quit()

parse_url

'Nt7QuZxczVWVGKr0uOzRr+U4Ch3AwiEt0Z3vtIQL1XNjH1ytHSHaaTfLEC0qHtrIDpB33aBdgmqSPr1JRVcNtE5y2Ht5/gfL802VqdTZGdc/Rd3xwjYcOE+hVaFv11XABkqv5Zg9VtOMG7NtWtI1JSXZLfB8G9Qs2lzojVbDRgFIhBN8LL2xRLdHXuvWd1+z9slZQzJ86SLtacP2CZPwT5UjW+0v38fzheLaHULdTxCNIj+Yv7E+OjqoWUSjbWIRGw5Fxdu70zf5wfvMZ/ckz6JtxVE3rfqaRBeOtT73kCog8KAhX1o8SFyd4G96TkpM1UTjkbMsWaERrrg22XySZ0/BzphaEmrCymXT9a1jQ7oG6uvwD2l6Yg2FlBXlw3BUKG0CE+nQCBxb6KkZLT/gO/AsjSQ4TVvQwYkBCSFXT/GYwZETdHxVtAYHTvN100MQuG4LYrgoVi/JHHvlHjKjxvx35Oun5pd755iVorlNKuwSjZKd/3xjRoM+8BCqmkHB7cGfU7r+bhwqM1FTwTm5CSTb9UtdaFLDbG5F0zprB7YC0scPDs/2idRncIosLSJQeNtsU/GcT+O2afDMapb9xIpA4FsXD0QRrb00xVCQGaAztbFzinFQWFpvnT5WAoG4yOOKyPLe/QTuDKKtChr+fYngOFU15CoTFOPDGtnAmgBtkMzwGyIJcvp+RfrVBVDoSr4UnzsH02KY4bIb7O2tfruD5aRP6TXUkPB5qWk13qejB6kihWtmOndom96+aIwTfZIKuUbFPPd3Ikz/+qZX1VtuNMRavvkJYDRe7hk2351jOiI2sL4dDDneZgcCXT3Fg3YXlWkjV4zGNwuuO02zIrgeWjd3SOLHZ6qUIjIg2gX/4exGNRPE8SZijK+euAsR+jev53MB0ZE=212FD3x19z9sWBHDJACbC00B75E'

In [4]:
# authorization
endpoint = 'https://api.tdameritrade.com/v1/oauth2/token'
headers = {"Content-Type":"application/x-www-form-urlencoded"}
payload = {'grant_type':'authorization_code',
           'access_type': 'offline',
           'code': parse_url,
           'client_id': client_id,
           'redirect_uri': 'https://127.0.0.1'}

# post data for token
authReply = requests.post(endpoint, headers=headers, data=payload)

# convert from json to dict
content = authReply.json()
content

{'access_token': '8EA2u/iUchV2umYPOnsxiCwAVZcfok6R9r8gH8KznJ+8EHX8T0isAhl2gJvz3qy4+eyJOSj2D/uwTojXaX4tjQk9DrhsayOQxdizyaqOJ1OZgCUUaTy7pF65igiz5YehK+cyaMX4kImBSSH7Z4YC+OjZiAaxU+X8z86KALlfVbSaKlsJ3ViZrih4wtqQf5Wpk/qk0ObN29aBbKRSTFeAoAtC3ErgOf1OqntgE7W+gDFV5x+3X8+xYQw/u4BeUuKxAG6N8mwssO2xkaogtlmUTgZmDyEZrEmI4Ad6Oxsp/d0rQs2myENfHIJ3xb0LE3bZhCMai5F89EKBdTBuyjIEUepvMKcYeZ4P3XCPIrNJJhB/rUS/5HvvLcHu0AIDc8fhbuLzyXz9TBzH2BH6dB2xj9l/BEmLhT2dmr0Wi+InSgFKcqQuf8ZqTTBBeSdUyPPHzv6vNhT2RiFOFmLmCItZhfXp21amTpfxml8wUxpmWDUvTbM/rOXjpA3If4n3P8l+MS7wWlXQz91b4Ipfy100MQuG4LYrgoVi/JHHvlyQn0M69LirNvESg7E3a4cPnMck9qj9kFmuVfo45w8kUyI8GftmfGAk1+Hc6yrBuTx1aXvBam5dzAEwLvAdHKvzxTK82O4GIWDIDQB7D+TQAcxp+82IBxvxhxWdmB4sCI3QW4fHQWOnN0Xksoh7bH5KzVTjVCVekWAo7yRARCH6PxXnlNUAHfTtSP48gEMaCsyi/wYuTvvLnmKjqjS1x3OwcJx9hLsvlxyFnRnIXBd4oHo1KBs6tfJB0kKHwFHWWF5AvpJkhWuQygeyP5MmQYDqx8qnEm4053LhmNYf3ATGxn+kiBj7mlcsenvVRgWRmojBBtDc+krmZDdEN9Ch7UmBwPsBEb40u7oJRUhu3t89cWklyVXunwee+ox4aCeXQ8IRpCI+FaRQHFB+6KA79h+J4ZkDjte9l+koPyWSxEtnDee+H3

In [5]:
# access token request header
access_token = content['access_token']
headers = {'Authorization': "Bearer {}".format(access_token)}

In [6]:
# accounts endpoint
endpoint = 'https://api.tdameritrade.com/v1/accounts'
content = requests.get(url = endpoint, headers = headers)
data = content.json()
display(data)

[{'securitiesAccount': {'type': 'MARGIN',
   'accountId': '489390427',
   'roundTrips': 0,
   'isDayTrader': False,
   'isClosingOnlyRestricted': False,
   'initialBalances': {'accruedInterest': 0.07,
    'availableFundsNonMarginableTrade': 17999.83,
    'bondValue': 0.0,
    'buyingPower': 39674.56,
    'cashBalance': 0.0,
    'cashAvailableForTrading': 0.0,
    'cashReceipts': 0.0,
    'dayTradingBuyingPower': 0.0,
    'dayTradingBuyingPowerCall': 0.0,
    'dayTradingEquityCall': 0.0,
    'equity': 26463.83,
    'equityPercentage': 100.0,
    'liquidationValue': 26463.83,
    'longMarginValue': 20888.4,
    'longOptionMarketValue': 0.0,
    'longStockValue': 20888.4,
    'maintenanceCall': 0.0,
    'maintenanceRequirement': 8464.0,
    'margin': 0.0,
    'marginEquity': 20888.4,
    'moneyMarketFund': 5575.43,
    'mutualFundValue': 0.0,
    'regTCall': 0.0,
    'shortMarginValue': 0.0,
    'shortOptionMarketValue': 0.0,
    'shortStockValue': 0.0,
    'totalCash': 0.0,
    'isInCall

In [7]:
account_id = data[0]['securitiesAccount']['accountId']
account_id

'489390427'